# Cleaning Wait Times Data

In [ ]:
import os
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv('../data/uss_wait_times.csv')

df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns={'Ride': 'ride', 'Date/Time': 'datetime', 'Wait Time': 'waittime'})
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter for entries from Q1 2023 to Q2 2024
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 6, 30, 23, 59, 59)

df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)].sort_values(by='datetime')

In [ ]:
# Park opens at 10am and typically closes at 9pm. We look into the entries recorded before 10am and after 10pm (to account for an hour's buffer after the park's closing time).
df_early = df[(df['datetime'].dt.hour < 10) & (df['waittime'] > 0)].groupby([df['datetime'].dt.month, df['datetime'].dt.day]).agg({'waittime': 'mean'})
df_late = df[(df['datetime'].dt.hour >
              21) & (df['waittime'] > 0)].groupby([df['datetime'].dt.month, df['datetime'].dt.day]).agg({'waittime': 'mean'})

print(df_early)
print(df_late)

In [ ]:
# Rides are open during Halloween Horror Night, which occurred between 29 Sept to 4 Nov 2023.
# As such, we keep "early" and "late" entries in the df, and exclude the rest, which may have been included due to errors.
df_waittimes = df[((df['datetime'].dt.hour >= 10) & (df['datetime'].dt.hour <= 21)) |
        ((df['datetime'].dt.hour < 10) & (df['waittime'] > 0) & (df['datetime'].dt.month != 2)) |
        ((df['datetime'].dt.hour > 21) & (df['waittime'] > 0) & (df['datetime'].dt.month == 9)) |
        ((df['datetime'].dt.hour > 21) & (df['waittime'] > 0) & (df['datetime'].dt.month == 10))]

# # Save the filtered data to CSV
# df_waittimes.to_csv("/content/waittimes_full.csv", index=False)

# Extracting Weather Data from Data.gov.sg API
URL: https://data.gov.sg/datasets/d_ce2eb1e307bda31993c533285834ef2b/view#tag/default/GET/twenty-four-hr-forecast


In [ ]:
import requests
import datetime
import csv
import time

In [ ]:
def is_raining(forecast_text):
    # Define keywords that indicate rain
    rain_keywords = ['Rain', 'Showers', 'Thundery Showers', 'Heavy Showers', 'Light Showers']
    return 1 if any(keyword in forecast_text for keyword in rain_keywords) else 0

def get_weather_data_south_region(start_date, end_date, output_file):
    url = "https://api-open.data.gov.sg/v2/real-time/api/twenty-four-hr-forecast"
    current_datetime = datetime.datetime.combine(start_date, datetime.time(0, 0))


    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['datetime', 'forecast', 'mean temperature (°C)', 'is raining'])

        while current_datetime <= datetime.datetime.combine(end_date, datetime.time(23, 59)):
            datetime_str = current_datetime.strftime("%Y-%m-%dT%H:%M:%S")
            params = {'date': datetime_str}

            try:
                response = requests.get(url, params=params)

                if response.status_code == 200:
                    data = response.json()
                    record_found = False
                    for record in data['data']['records']:
                        general_forecast = record.get('general', {})
                        temperature = general_forecast.get('temperature', {})
                        low_temp = temperature.get('low', None)
                        high_temp = temperature.get('high', None)

                        mean_temp = (low_temp + high_temp) / 2 if low_temp and high_temp else 'N/A'

                        for period in record.get('periods', []):
                            if 'south' in period['regions'] and not record_found:
                                weather_info = period['regions']['south']
                                forecast_text = weather_info['text']
                                raining = is_raining(forecast_text)

                                writer.writerow([datetime_str, forecast_text, mean_temp, raining])
                                record_found = True  # Ensure we only write one record per interval

                    if not record_found:

                        writer.writerow([datetime_str, 'N/A', 'N/A', 0])

                else:
                    raise Exception(f"Failed with status code {response.status_code}")

            except Exception as e:
                print(f"Failed to retrieve data for {datetime_str}, using 'N/A' values: {str(e)}")
                writer.writerow([datetime_str, 'N/A', 'N/A', 0])

            current_datetime += datetime.timedelta(minutes=30)


start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2024, 6, 30)


output_file = '../data/raw/sentosa_region_weather_data.csv'

get_weather_data_south_region(start_date, end_date, output_file)

print(f"Weather data saved to {output_file}")


Failed to retrieve data for 2023-01-02T08:00:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T08:30:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T09:00:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T09:30:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T10:00:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T10:30:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T11:00:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T11:30:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T12:00:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 2023-01-02T12:30:00, using 'N/A' values: Failed with status code 404
Failed to retrieve data for 20

In [ ]:
import pandas as pd

waittimes = df_waittimes.copy()

waittimes.set_index('datetime', inplace=True)

average_wait_time = waittimes['waittime'].resample('30T').mean().reset_index()

average_wait_time.columns = ['datetime', 'average_waittime']
average_wait_time = average_wait_time[average_wait_time['average_waittime'] != 0]

print(average_wait_time)

                 datetime  average_waittime
2     2023-01-02 11:00:00          7.916667
3     2023-01-02 11:30:00         27.272727
4     2023-01-02 12:00:00         37.232143
5     2023-01-02 12:30:00         47.822581
6     2023-01-02 13:00:00         45.135135
...                   ...               ...
26173 2024-06-30 16:30:00         23.154762
26174 2024-06-30 17:00:00         22.440476
26175 2024-06-30 17:30:00         19.761905
26176 2024-06-30 18:00:00         14.464286
26177 2024-06-30 18:30:00         14.523810

[22572 rows x 2 columns]


<ipython-input-17-70a12a3984a3>:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  average_wait_time = waittimes['waittime'].resample('30T').mean().reset_index()


# Combining Wait Time and Weather Data

In [ ]:
weather = pd.read_csv('../data/raw/sentosa_region_weather_data.csv', parse_dates=['datetime'])
combined=pd.merge(average_wait_time, weather, on='datetime', how='inner')


In [ ]:
combined
combined.to_csv('../data/processed/uss_waittime_and_weather.csv', index=False)